In [1]:
import numpy as np
import pandas as pd

from UpdateDB_ref import Endpoint_annotation as ep


*** CompoundDB module found. Will check the synonyms table to resolve CAS. ***



In [2]:
ep_refactored = ep.Endpoint(host='gea', dbname='cii_v3_test_refactored', user='postgres', password='DBAdmin')

In [3]:
endpoints_ref = ep_refactored.get_endpoints()

In [4]:
ep_refactored.get_total_annotations_per_endpoint(endpoints_ref)

/home/emarch/miniconda3/envs/inditex/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/home/emarch/miniconda3/envs/inditex/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,Endpoints,YES,Pending,NO,No information
0,subs_id,0.0,0.0,0.0,0.0
1,cmr,1401.0,39.0,489.0,2765.0
2,pbt,44.0,0.0,385.0,4265.0
3,vpvb,49.0,0.0,385.0,4260.0
4,endocrine_disruptor,158.0,0.0,0.0,4536.0
5,sensitiser,68.0,72.0,0.0,1287.0
6,chem_id,0.0,0.0,0.0,0.0


In [5]:
endpoint_pos = pd.read_sql_query("""select pea.annotation_id, rn.names, pea.endpoint 
from positive_endpoint_annotations pea
left join regulation_names rn on rn.id = pea.annotation_id
order by pea.id """, ep_refactored.conn)

In [6]:
endpoint_dict = {'CMR':endpoint_pos.loc[endpoint_pos['endpoint'] == 'cmr','names'].tolist(),
                 'PBT':endpoint_pos.loc[endpoint_pos['endpoint'] == 'pbt','names'].tolist(),
                 'vPvB':endpoint_pos.loc[endpoint_pos['endpoint'] == 'vpvb','names'].tolist(),
                 'Endocrine Disruptor':endpoint_pos.loc[endpoint_pos['endpoint'] == 'endocrine disruptor','names'].tolist()}

In [7]:
chem_ids = ep_refactored.get_chem_id_dataframe()

In [8]:
subs = ep_refactored.get_substances()

In [9]:
len(chem_ids)

5672

In [10]:
len(subs)

4694

In [11]:
len(subs[['class_name','preferred_name','mol_formula']].drop_duplicates())

4611

In [12]:
len(chem_ids.loc[chem_ids['chem_type_id'] == 1, :])

4694

In [13]:
subs.loc[subs['preferred_name'] == "Silicon dioxide"].sort_values(by='id')

,id,class_name,preferred_name,mol_formula
42,919,Textile Chemicals,Silicon dioxide,O2Si
698,4639,Textile Chemicals,Silicon dioxide,O2Si


In [14]:
rep_subs = subs.loc[(subs['id'] > 4634) &
                    (~subs['preferred_name'].isna()), 'preferred_name'].unique()

In [15]:
repeated_subs = subs.loc[subs['preferred_name'].isin(rep_subs)].sort_values(by='preferred_name')

In [16]:
# repeated_subs.to_excel('repeated_subs_cii.xlsx')

In [17]:
chem_ids_unique = chem_ids.loc[chem_ids['chem_type_id'] == 1, 'id'].unique()

In [18]:
%%time
chemid_endpoint = ep_refactored.get_annotations_for_chem_id(chem_ids_unique, endpoint_dict)

CPU times: user 2min 19s, sys: 1.71 s, total: 2min 21s
Wall time: 11min 32s


In [19]:
len(chemid_endpoint)

4694

In [20]:
chemid_endpoint.sort_values(by='chem_id').head()

,chem_id,CMR,PBT,vPvB,Endocrine Disruptor
24,1.0,YES,NO,NO,No information
25,2.0,No information,No information,No information,No information
26,5.0,YES,NO,NO,YES
27,7.0,YES,NO,NO,No information
28,9.0,YES,No information,No information,No information


In [21]:
len(endpoints_ref)

4694

In [22]:
endpoints_ref.sort_values(by='chem_id')

,id,subs_id,cmr,pbt,vpvb,endocrine_disruptor,sensitiser,chem_id
0,0,1,YES,NO,NO,No information,YES,1
4652,4652,1,YES,NO,NO,No information,YES,2
1,1,2,YES,NO,NO,YES,No information,5
2,2,3,YES,NO,NO,No information,No information,7
3,3,4,YES,No information,No information,No information,No information,9
4,4,5,YES,No information,No information,No information,No information,11
5,5,6,YES,No information,No information,No information,No information,13
6,6,7,YES,No information,No information,No information,No information,15
7,7,8,YES,No information,No information,No information,No information,17
8,8,9,YES,No information,No information,No information,No information,19


### Here it should come the check part
After getting the endpoint annotations per substance, and at the same time we update the database with them, the check function should come.
Before updating the DB, the check function should see if there's a substance id in the table.
If no, adds the new substance with the annotations.
If there is, then it checks the annotations and applies the filters we've spoken

In [23]:
# ep.add_annotations_to_database(subs_endpoint)

### Total annotation check
This only counts the total number of annotations per endpoint we have

In [24]:
ep_refactored.get_total_annotations_per_endpoint(chemid_endpoint)

,Endpoints,YES,Pending,NO,No information
0,CMR,1362.0,39.0,0.0,3293.0
1,PBT,38.0,0.0,357.0,4299.0
2,vPvB,43.0,0.0,357.0,4294.0
3,Endocrine Disruptor,148.0,0.0,0.0,4546.0
